# Next cell is only applicable in colab 

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# Necessário ter uma pasta FIRELOC_DATA na Drive, com os ficheiros (weights, train.txt...) que estão no GitHub

Mounted at /content/drive


In [4]:
# verify CUDA
# para funcionar com GPU, Runtime->Change runtime type para GPU
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [5]:
# path para weights e train.txt (que tem o path das imagens a testar)
# Se estivermos a executar no Colab
path_fireloc = "/content/drive/MyDrive/"
# Se estivermos a executar localmente, criamos uma pasta FIRELOC_DATA na mesma diretoria que "darknet"
#path_fireloc = "../"

# path para cfg e data
# colab
path_cfg = "/content/drive/MyDrive/YOLOv4/"
# local
# path_cfg = ""

In [6]:
# fork do repositório de AlexeyAB com alterações do LIME
!git clone https://github.com/AntMorais/yolime

Cloning into 'darknet'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 14667 (delta 7), reused 19 (delta 6), pack-reused 14642
Receiving objects: 100% (14667/14667), 13.71 MiB | 11.04 MiB/s, done.
Resolving deltas: 100% (9960/9960), done.


In [7]:
# install lime
!pip install lime

     |████████████████████████████████| 276kB 10.9MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp36-none-any.whl size=283846 sha256=2d0e4549c8122afd6ee0b4fade25bd4e413010786ff81d69c4262c34dd427b03
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime


In [8]:
%cd darknet
# se utilizar make, o nome do shared file que se utiliza tem de ser libdarknet.so, no caso de utilizar build.sh o nome é libdarknetd.so
# Makefile:
# opencv = 0 (mas pode estar a 1, podemos experimentar e ver se da alguma coisa diferente)
# gpu = 1
# cudnn = 1
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DDEBUG -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -DDEBUG -Ofast -fopenmp -DGPU -DCUDNN -I/usr/local/cudnn/include -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DDEBUG -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -DDEBUG -Ofast -fopenmp -DGPU -DCUDNN -I/usr/local/cudnn/include -fPIC -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:249:21: warning: unused variable ‘n’ [-Wunused-variable]
                 int n = _write(client, outputbuf, outlen);
                     ^
./src/http_stream.cpp: In function ‘void set_track_id(detection*, int, float, float, float, int, int, int

In [9]:
# setup dos dados de treino
%cd darknet
!cp /content/drive/MyDrive/YOLOv4/img_treino_mais_teste.zip ../
!unzip ../img_treino_mais_teste.zip -d ./data/
!cp /content/drive/MyDrive/YOLOv4/obj.names ./data
!cp /content/drive/MyDrive/YOLOv4/obj.data  ./data 
!cp /content/drive/MyDrive/YOLOv4/generate_train.py ./
!python generate_train.py

[Errno 20] Not a directory: 'darknet'
/content/darknet
Archive:  ../img_treino_mais_teste.zip
   creating: ./data/obj/
  inflating: ./data/obj/20190515_091756.jpg  
 extracting: ./data/obj/20190515_091756.txt  
  inflating: ./data/obj/20190515_091809.jpg  
 extracting: ./data/obj/20190515_091809.txt  
  inflating: ./data/obj/20190515_091855.jpg  
 extracting: ./data/obj/20190515_091855.txt  
  inflating: ./data/obj/20190515_091902.jpg  
 extracting: ./data/obj/20190515_091902.txt  
  inflating: ./data/obj/20190515_091918.jpg  
 extracting: ./data/obj/20190515_091918.txt  
  inflating: ./data/obj/20190515_091923.jpg  
 extracting: ./data/obj/20190515_091923.txt  
  inflating: ./data/obj/20190515_092016.jpg  
 extracting: ./data/obj/20190515_092016.txt  
  inflating: ./data/obj/20190515_092022.jpg  
 extracting: ./data/obj/20190515_092022.txt  
  inflating: ./data/obj/20190515_092028.jpg  
 extracting: ./data/obj/20190515_092028.txt  
  inflating: ./data/obj/20190515_092248.jpg  
 extrac

To choose an image, write its relative path in data/train.txt


```
data/obj/IMAGENAME.jpg
```

Example:

```
data/obj/20190515_113930.jpg
```



In [ ]:
import argparse
import os
import glob
import random
import time
import cv2
import numpy as np
import darknet
import darknet_images


# Sempre que alteramos uma imagem, (pomos um path no train.txt) temos de correr tudo o que esta em baixo disto

# LOAD and CONFIG

# No caso de querermos correr a rede default (para testar por ex.)
#txt_input = path_fireloc + "FIRELOC_DATA/train.txt"
#weights = path_fireloc + "FIRELOC_DATA/yolov4.weights"
#config_file = "cfg/yolov4.cfg"
#data_file = "cfg/coco.data"

# No caso de querermos a rede FireLoc
txt_input = "data/train.txt"
weights = path_fireloc + "FIRELOC_DATA/yolov4_treino_maisteste_5000.weights"
config_file = path_cfg + "yolov4_treino_416.cfg"
data_file = "data/obj.data"
# outras configurações
batch_size = True
dont_show = True
ext_output = True
save_labels = True
thresh = 0.25


images = darknet_images.load_images(txt_input)
image_name = images[0]
network, class_names, class_colors = darknet.load_network(
        config_file,
        data_file,
        weights,
        batch_size
)

# pick class index out of the top 5 classes
# in the dog.jpg example:
# 0 is plant
# 1 is truck
# 2 is bicycle
# 3 is dog
# how many samples to use in explain_instance
num_samples = 1000
# how many features maximum can be used in the explanation (default is 100000)
num_features = 1000000



#---------------------------------------------------------------------------------------------

width = darknet.network_width(network)
height = darknet.network_height(network)
image = cv2.imread(image_name)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_resized = cv2.resize(image_rgb, (width, height),
                            interpolation=cv2.INTER_LINEAR)



def grey_out(image, x, y, w, h):
    # input: ndarray returned by imread, height and width
    #Create an array of object rect which represents the region of interest
    rect = [[x-w/2,y+h/2], [x-w/2,y-h/2], [x+w/2,y-h/2],[x+w/2,y+h/2]]
    mask = np.array([rect], dtype=np.int32)

    #Create a new array filled with zeros, size equal to size of the image to be filtered
    image2 = np.zeros((width, height), np.int8)

    cv2.fillPoly(image2, [mask],255)    
    maskimage2 = cv2.inRange(image2, 1, 255)
    out = cv2.bitwise_and(image, image, mask=maskimage2)
    #cv2.imshow("img",out)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return out



# so funciona para uma imagem
def lime_classification_function(image_numpy):

    probabilities = np.zeros((len(image_numpy), len(class_names)))
    for i in range(len(image_numpy)):
        img = image_numpy[i]
        image, detections = darknet_images.image_detection_lime(
                img, network, class_names,
                class_colors, thresh
                )

        if save_labels:
            darknet_images.save_annotations(image_name, image, detections, class_names)
        #darknet.print_detections(detections, ext_output)
        if not dont_show:
            cv2.imshow('Inference', image)
            if cv2.waitKey() & 0xFF == ord('q'):
                return

        with open("probabilityArray.txt","r") as prob_array:
            lines = [float(line.rstrip()) for line in prob_array]
            probabilities[i] = lines
    return probabilities


def get_most_confident_bbox(class_name):
    # First run through Yolo to check if
    image_original, detections = darknet_images.image_detection(
                image_name, network, class_names,
                class_colors, thresh
                )
    darknet.print_detections(detections, ext_output)
    # take the detection that had the most confidence and write the coordinates to a txt file
    #detections is a list of tuples (class, confidence,(coordinates))
    picked_class = -1
    for i in range(len(detections)):
        _tuple = detections[i]
        if _tuple[0] == class_name:
            coordinates = _tuple[2]
            with open("coordinates.txt","w") as coord_file:
                # we convert the coordinates to relative because that's how it is used in the C files
                coordinates_rel = darknet_images.convert2relative(image_original,coordinates)
                for _coord in coordinates_rel:
                    coord_file.write(str(_coord)+"\n")
                picked_class = i
                # we dont break here because we want the detection with the most confidence
    if picked_class == -1:
        print("There is no " + class_name + " in this image!")
    return picked_class, detections


In [ ]:



class_name = 'Smoke'
picked_class, coordinates = get_most_confident_bbox(class_name)



In [ ]:
#!./darknet detector test data/obj.data /content/drive/MyDrive/YOLOv4/yolov4_treino_416.cfg /content/drive/MyDrive/FIRELOC_DATA/yolov4_treino_maisteste_5000.weights -ext_output -dont_show -out result.json < data/train.txt

In [ ]:


# pass the image, the height and width
greyed_out_image = grey_out(image_resized, coordinates[picked_class][2][0], coordinates[picked_class][2][1],\
                            coordinates[picked_class][2][2], coordinates[picked_class][2][3]
                            )


In [ ]:

from google.colab.patches import cv2_imshow
cv2_imshow(cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB))

In [ ]:


from lime import lime_image
#Objeto do tipo LimeImageExplainer
explainer = lime_image.LimeImageExplainer()
#Objeto do tipo ImageExplanation
explanation = explainer.explain_instance(np.array(greyed_out_image), 
                                         lime_classification_function, # classification function
                                         top_labels=5, 
                                         hide_color=0, 
                                         num_samples=num_samples,
                                         num_features=num_features) # number of images that will be sent to classification function
    



In [ ]:
# mostra a imagem que acabámos de detetar

import cv2
img = cv2.imread(image_name)
from google.colab.patches import cv2_imshow
cv2_imshow(cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB))

In [ ]:


def show_bbox_centers(coordinates):

    import matplotlib.pyplot as plt
    #mostrar centros das bounding boxes 
    xx = []
    yy = []
    for class_coord in coordinates:
        xx.append(class_coord[2][0])
        yy.append(class_coord[2][1])
    plt.scatter(xx,\
            yy,\
            marker='o', color="red")
    
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0],\
    positive_only=False, num_features=5\
    ,hide_rest=False)
img_boundry2 = mark_boundaries(temp/255.0, mask)
plt.imshow(img_boundry2)
show_bbox_centers(coordinates)
plt.show()

In [ ]:

def crop_img(img, new_filename, coordinates, picked_class):
    [center_x, center_y, width, height] = [coord for coord in coordinates[picked_class][2]]
    top_left_x = round(center_x - width/2)
    top_left_y = round(center_y - height/2)
    print(top_left_x)
    print(top_left_y)
    print(height)
    print(width)
    if top_left_x < 0:
        top_left_x = 0
    if top_left_y < 0:
        top_left_y = 0
    crop_img = img[top_left_y:top_left_y+round(height), top_left_x:top_left_x+round(width)]
    plt.imshow(crop_img)
    plt.show()
    # guardo a nova imagem num ficheiro
    cv2.imwrite(new_filename,crop_img)



crop_img(img_boundry2, "explanationHere.jpg", coordinates, picked_class)